In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = 'HW-25'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data['CPORE'] = well_data['CORE_POR'] / 100
well_data['CPERM'] = well_data['CORE_PERM']
well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'], inplace=True)

# Quick PP - Single Well Interpretation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter import plotly_log, neutron_density_xplot
from quick_pp.utils import zone_flagging, min_max_line

## Estimate Litholoy

In [ ]:
# Define parameters
args = {
    'litho_model': 'carb',
    'dry_calc_point': (.0, 2.71),
    # 'dry_dolo_point': (.0, 2.81),
    'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 2e5,
    'sw_m': 2.2,
    'sw_n': 2,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

# Clean up data
well_data = badhole_flagging(well_data)
# well_data = mask_outside_threshold(well_data, fill=True)

vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=150)
carb_model = Carbonate(**args)
vclw, vcalc, vdolo = carb_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, normalize= True, carbonate_type=carbonate_type
)
args.update(carb_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Set skip HC correction or not
skip_hc_correction = False
if skip_hc_correction:
    nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']
else:
    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )
    # Estimate lithology
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        nphihc, rhobhc,  gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'], 
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )

In [ ]:
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], dry_min1_point=args['dry_calc_point'], **args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, dry_min1_point=args['dry_calc_point'], **args)

## Estimate Porosity

In [ ]:
# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhobhc, model='carb',
    dry_min1_point=args['dry_calc_point'],
    dry_clay_point=args['dry_clay_point'],
)

# PHID needs unnormalized lithology
vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
    nphihc, rhobhc, gr=well_data['GR'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, normalize= False, carbonate_type=carbonate_type
)
rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
phid = density_porosity(rhobhc, rho_ma)

plt.figure(figsize=(20, 2))
plt.plot(phit, label='PHIT')
plt.plot(phid, label='PHID')
plt.legend()

## Predict Permeability and Rock Typing

In [ ]:
import pickle
from quick_pp.rock_type import calc_r35_perm
from quick_pp.core_calibration import perm_transform

poroperm_params = {
    1: (6800, 1.9),
    2: (1400, 2.0),
    3: (110, 2.0),
    4: (17, 2.0)
}

input_features = ['GR', 'NPHI', 'RHOB', 'RT']
# Predict ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\rt_model.qppm', 'rb') as file:
    rt_model = pickle.load(file)
rock_flag_ml = rt_model.predict(well_data[input_features])

# Predict PERM based on predicted rock type
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_ml = perm_transform(phit, perm_a_ml, perm_b_ml)

# Predict PERM based on R35 model
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)

r35_ml = 10**(r35_model.predict(well_data[input_features]))
perm_r35_ml = calc_r35_perm(r35_ml, phit)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, perm_ml, label='Perm ML')
plt.plot(well_data.DEPTH, perm_r35_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

## Estimate Water Saturation

In [ ]:
from quick_pp.saturation import estimate_rw_temperature_salinity, archie_saturation, estimate_temperature_gradient
from quick_pp.core_calibration import sw_shf_leverett_j

# Estimate log derived water saturation
water_salinity = args['sw_water_salinity']
m = args['sw_m']
temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
swt = archie_saturation(well_data['RT'], rw, phit, m=m)
swt = swt.clip(0, 1)

# Estimate SHF
shf_params = {
    1: (.007, 1.2),
    2: (.012, 1.29),
    3: (.017, 1.34),
    4: (.018, 1.13),
    5: (.016, 1.27),
    6: (.022, 1.15),
    7: (.036, .49),
    8: (.015, 1.13),
    9: (0, .1)
}

ift = 32
theta = 30
ghc = .837
gw = 1.0

fwl = 8380

a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[0])
b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[1])
shf = sw_shf_leverett_j(
    perm_r35_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
    fwl=fwl, ift=ift, theta=theta, a=a, b=b
)

plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], swt, label='SWT')
plt.plot(well_data['DEPTH'], shf, label='SHF')
plt.ylim(0, 1)
plt.figure(figsize=(15, 1.5))
plt.plot(well_data['DEPTH'], rw, label='RW')
plt.yscale('log')
plt.legend()

## Update Data

In [ ]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['VCALC'] = vcalc
well_data['VDOLO'] = vdolo
well_data['VCLW'] = vclw
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['M'] = m
well_data['SWT'] = swt
well_data['SHF'] = shf
well_data['PERM'] = perm_r35_ml
well_data['VHC'] = phit * (1 - swt)

In [ ]:
from quick_pp.ressum import calc_reservoir_summary

# Estimate reservoir summary
well_data['ZONES'] = 'ALL'
ressum_df = calc_reservoir_summary(
    well_data.DEPTH, vclw, phit, swt, perm_r35_ml,
    zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

In [ ]:
# Plot the results
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))